<a href="https://colab.research.google.com/github/cosmo3769/Quantized-LLMs/blob/main/notebooks/quantize_CodeLlama_7b_GGUF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Mount drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Install llama.cpp

In [2]:
# Install llama.cpp
!git clone https://github.com/ggerganov/llama.cpp
!cd llama.cpp && git pull && make clean && LLAMA_CUBLAS=1 make
!pip install -r llama.cpp/requirements.txt

Cloning into 'llama.cpp'...
remote: Enumerating objects: 19468, done.
remote: Counting objects: 100% (6016/6016), done.
remote: Compressing objects: 100% (320/320), done.
remote: Total 19468 (delta 5865), reused 5736 (delta 5695), pack-reused 13452
Receiving objects: 100% (19468/19468), 22.85 MiB | 21.95 MiB/s, done.
Resolving deltas: 100% (13680/13680), done.
Already up to date.
I ccache not found. Consider installing it for faster compilation.
I llama.cpp build info: 
I UNAME_S:   Linux
I UNAME_P:   x86_64
I UNAME_M:   x86_64
I CFLAGS:    -I. -Icommon -D_XOPEN_SOURCE=600 -D_GNU_SOURCE -DNDEBUG  -std=c11   -fPIC -O3 -Wall -Wextra -Wpedantic -Wcast-qual -Wno-unused-function -Wshadow -Wstrict-prototypes -Wpointer-arith -Wmissing-prototypes -Werror=implicit-int -Werror=implicit-function-declaration -pthread -march=native -mtune=native -Wdouble-promotion 
I CXXFLAGS:  -std=c++11 -fPIC -O3 -Wall -Wextra -Wpedantic -Wcast-qual -Wno-unused-function -Wmissing-declarations -Wmissing-noreturn -

In [ ]:
import os
os.kill(os.getpid(), 9)

In [1]:
!mkdir CodeLlama-7b-hf

# Choose Model

In [2]:
# Variables
MODEL_PATH = "drive/MyDrive/CodeLlama-7b-hf"
QUANTIZATION_METHOD = "q4_k_m"

# Constants
MODEL_NAME = "CodeLlama-7b-hf"

# Quantize Model

In [3]:
# Convert to fp16
fp16 = f"{MODEL_NAME}/{MODEL_NAME.lower()}.fp16.bin"
!python llama.cpp/convert.py {MODEL_PATH} --outtype f16 --outfile {fp16}

Loading model file drive/MyDrive/CodeLlama-7b-hf/pytorch_model-00001-of-00003.bin
Loading model file drive/MyDrive/CodeLlama-7b-hf/pytorch_model-00001-of-00003.bin
Loading model file drive/MyDrive/CodeLlama-7b-hf/pytorch_model-00002-of-00003.bin
Loading model file drive/MyDrive/CodeLlama-7b-hf/pytorch_model-00003-of-00003.bin
params = Params(n_vocab=32016, n_embd=4096, n_layer=32, n_ctx=16384, n_ff=11008, n_head=32, n_head_kv=32, n_experts=None, n_experts_used=None, f_norm_eps=1e-05, rope_scaling_type=None, f_rope_freq_base=1000000, f_rope_scale=None, n_orig_ctx=None, rope_finetuned=None, ftype=<GGMLFileType.MostlyF16: 1>, path_model=PosixPath('drive/MyDrive/CodeLlama-7b-hf'))
Found vocab files: {'tokenizer.model': PosixPath('drive/MyDrive/CodeLlama-7b-hf/tokenizer.model'), 'vocab.json': None, 'tokenizer.json': PosixPath('drive/MyDrive/CodeLlama-7b-hf/tokenizer.json')}
Loading vocab file 'drive/MyDrive/CodeLlama-7b-hf/tokenizer.model', type 'spm'
Vocab info: <SentencePieceVocab with 32

In [4]:
# Quantize the model
qtype = f"{MODEL_NAME}/{MODEL_NAME.lower()}.{QUANTIZATION_METHOD.upper()}.gguf"
!./llama.cpp/quantize {fp16} {qtype} {QUANTIZATION_METHOD}

ggml_init_cublas: GGML_CUDA_FORCE_MMQ:   no
ggml_init_cublas: CUDA_USE_TENSOR_CORES: yes
ggml_init_cublas: found 1 CUDA devices:
  Device 0: Tesla T4, compute capability 7.5, VMM: yes
main: build = 2295 (87c91c07)
main: built with cc (Ubuntu 11.4.0-1ubuntu1~22.04) 11.4.0 for x86_64-linux-gnu
main: quantizing 'CodeLlama-7b-hf/codellama-7b-hf.fp16.bin' to 'CodeLlama-7b-hf/codellama-7b-hf.Q4_K_M.gguf' as Q4_K_M
llama_model_loader: loaded meta data with 21 key-value pairs and 291 tensors from CodeLlama-7b-hf/codellama-7b-hf.fp16.bin (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = MyDrive
llama_model_loader: - kv   2:                       llama.context_length u32              = 16384
llama_model_loader: - kv   3:               

# Run inference

In [5]:
import os

model_list = [file for file in os.listdir(MODEL_NAME) if "gguf" in file]

prompt = input("Enter your prompt: ")
chosen_method = input("Name of the model (options: " + ", ".join(model_list) + "): ")

# Verify the chosen method is in the list
if chosen_method not in model_list:
    print("Invalid name")
else:
    qtype = f"{MODEL_NAME}/{MODEL_NAME.lower()}.{QUANTIZATION_METHOD.upper()}.gguf"
    !./llama.cpp/main -m {qtype} -n 128 --color -ngl 35 -p "{prompt}"

Enter your prompt: Write python function to add two integers.
Name of the model (options: codellama-7b-hf.Q4_K_M.gguf): codellama-7b-hf.Q4_K_M.gguf
Log start
main: build = 2295 (87c91c07)
main: built with cc (Ubuntu 11.4.0-1ubuntu1~22.04) 11.4.0 for x86_64-linux-gnu
main: seed  = 1709169607
ggml_init_cublas: GGML_CUDA_FORCE_MMQ:   no
ggml_init_cublas: CUDA_USE_TENSOR_CORES: yes
ggml_init_cublas: found 1 CUDA devices:
  Device 0: Tesla T4, compute capability 7.5, VMM: yes
llama_model_loader: loaded meta data with 22 key-value pairs and 291 tensors from CodeLlama-7b-hf/codellama-7b-hf.Q4_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = MyDrive
llama_model_loader: - kv   2:                       llama.context_length u3

In [6]:
import os

model_list = [file for file in os.listdir(MODEL_NAME) if "gguf" in file]

prompt = input("Enter your prompt: ")
chosen_method = input("Name of the model (options: " + ", ".join(model_list) + "): ")

# Verify the chosen method is in the list
if chosen_method not in model_list:
    print("Invalid name")
else:
    qtype = f"{MODEL_NAME}/{MODEL_NAME.lower()}.{QUANTIZATION_METHOD.upper()}.gguf"
    !./llama.cpp/main -m {qtype} -n 128 --color -ngl 35 -p "{prompt}"

Enter your prompt: Write a python function to output the fibonacci numbers till 100.
Name of the model (options: codellama-7b-hf.Q4_K_M.gguf): codellama-7b-hf.Q4_K_M.gguf
Log start
main: build = 2295 (87c91c07)
main: built with cc (Ubuntu 11.4.0-1ubuntu1~22.04) 11.4.0 for x86_64-linux-gnu
main: seed  = 1709169687
ggml_init_cublas: GGML_CUDA_FORCE_MMQ:   no
ggml_init_cublas: CUDA_USE_TENSOR_CORES: yes
ggml_init_cublas: found 1 CUDA devices:
  Device 0: Tesla T4, compute capability 7.5, VMM: yes
llama_model_loader: loaded meta data with 22 key-value pairs and 291 tensors from CodeLlama-7b-hf/codellama-7b-hf.Q4_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = MyDrive
llama_model_loader: - kv   2:                       